# 01. Put historical data in df

Data were saved as json files, now put them in a single df

- Ignore folders only containing a "_SUCCESS" file but no data
- Remove duplicate rows in df that arise because of getting the same data from arXiv multiple times

In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, from_json, schema_of_json, regexp_replace, udf
from pylatexenc.latex2text import LatexNodes2Text

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# count = sum(
#     1 for name in os.listdir("spark/notebooks")
#     if name.startswith("saved_data-") and os.path.isdir(os.path.join("spark/notebooks", name))
# )

# print(f"Number of folders starting with 'saved_data': {count}")

In [ ]:
def has_json_file(folder_path):
    try:
        files = [f for f in os.listdir(folder_path) if "SUCCESS" not in f]

        if not files:
            return False

        else:
            return True

    except Exception:
        return False

In [ ]:
has_json_file("spark/notebooks/saved_data-1743417480000")

In [ ]:
has_json_file("spark/notebooks/saved_data-1743416280000")

In [ ]:
data_folders = []

for folder in os.listdir("spark/notebooks"):
    folder_path = os.path.join("spark/notebooks", folder)
    if folder.startswith("saved_data") and os.path.isdir(folder_path):
        if has_json_file(folder_path):
            data_folders.append(folder_path)

In [ ]:
print(data_folders[0])

In [ ]:
len(data_folders)

In [ ]:
first_file = spark.read.text(data_folders[0]).limit(1).collect()[0][0]
first_file

In [ ]:
schema = schema_of_json(first_file)

In [ ]:
df_raw = spark.read.text(data_folders)
df_parsed = df_raw.withColumn("data", from_json(col("value"), schema)).select("data.*")
df_parsed_deduped = df_parsed.drop_duplicates(["title"])

df_parsed_deduped.show()

In [ ]:
df_parsed_deduped.count()

In [ ]:
df_parsed_deduped.write.mode("overwrite").parquet("data/df_all_deduped")

In [ ]:
df_parsed_deduped = spark.read.parquet("data/df_all_deduped")

In [ ]:
df_parsed_deduped.show()

In [ ]:
df_parsed_deduped.filter(col("published") == "2025-03-27T09:58:07Z").select(col("summary")).collect()

In [ ]:
def latex_to_text(s):
    return LatexNodes2Text().latex_to_text(s)

In [ ]:
def latex_to_text(s):
    if s is None:
        return None
    try:
        converted = LatexNodes2Text().latex_to_text(s)
        return converted.replace("\n", " ").replace("\r", " ").replace("^", "")
    except Exception:
        return None

In [ ]:
latex_udf = udf(latex_to_text, StringType())

df_cleaned = df_parsed_deduped.withColumn("title", latex_udf(col("title"))).withColumn("summary", latex_udf(col("summary")))
df_cleaned.show()

In [ ]:
df_cleaned.filter(col("published") == "2025-03-27T09:58:07Z").select(col("summary")).collect()

In [ ]:
df_cleaned.write.mode("overwrite").parquet("data/df_all_cleaned")

# 02 Scrape arXiv categories

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://arxiv.org/category_taxonomy'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# soup

In [ ]:
# soup.find_all(['h2', 'h3', 'h4'])

In [ ]:
categories = []

for header in soup.find_all(['h2', 'h3', 'h4']):
    if header.name == 'h2':
        group_name = header.get_text(strip=True)
    elif header.name == 'h3':
        archive_name = header.get_text(strip=True)
    elif header.name == 'h4':
        category_info = header.get_text(strip=True)
        # Extract category ID and name
        if '(' in category_info and ')' in category_info:
            category_id = category_info.split('(')[-1].strip(')')
            category_name = category_info.split('(')[0].strip()
        else:
            category_id = ''
            category_name = category_info
        categories.append({
            'group': group_name,
            'archive': archive_name,
            'category_id': category_id,
            'category_name': category_name
        })

In [ ]:
df = pd.DataFrame(categories)
df = df[df.group != "Group Name"]

df.head(20)

In [ ]:
df[df.category_name.str.contains('hep')]

In [ ]:
df.group.unique()

In [ ]:
df.to_parquet("data/arxiv_categories.parquet.gzip")

# 03 Create training and test set

In [ ]:
from pyspark.sql.functions import col, split, when, collect_set, concat, concat_ws, lit, regexp_replace, rand

In [ ]:
df_pub = spark.read.parquet("data/df_all_cleaned")
df_pub = df_pub.withColumn("level1_category", split(df_pub["main_category"], "\.")[0]) \
    .withColumn("level2_category", split(df_pub["main_category"], "\.")[1])
df_pub.show()

In [ ]:
df_categories = spark.read.parquet("data/arxiv_categories.parquet.gzip")
df_categories = df_categories.withColumn("level1_category", split(df_categories["category_name"], "\.")[0]) \
    .withColumn("level2_category", split(df_categories["category_name"], "\.")[1])
df_categories.show()

In [ ]:
df_level1_categories = df_categories.select(col('group'), col('level1_category'), col('archive')).drop_duplicates().sort(col('group'))
df_level1_categories = df_level1_categories.withColumn("subgroup",
    when(col("level1_category").isin("cs", "econ", "eess", "math", "q-bio", "q-fin", "stat"), col("group")).otherwise(concat(col("group"), lit(": "), col("archive")))
    ).drop('archive')
df_level1_categories = df_level1_categories.withColumn("subgroup", regexp_replace(col("subgroup"), r"\(.*?\)", "")
)

df_level1_categories.show()

In [ ]:
row = df_level1_categories.agg(collect_set("subgroup").alias("subgroups")).collect()[0]

print(row['subgroups'])

In [ ]:
df_labeled = df_pub.join(df_level1_categories, on = "level1_category", how = "left")
df_labeled.show()

In [ ]:
# concatenate title and summary; keep only that column and the label
df_relevant = df_labeled.withColumn("text", concat_ws(". ", col("title"), col("summary"))).select(col('text'), col('group'))
df_relevant.show()

In [ ]:
# shuffle the data
df_shuffled = df_relevant.orderBy(rand())

In [ ]:
# split into train/test (e.g., 80% train, 20% test)
train_df, test_df = df_shuffled.randomSplit([0.8, 0.2], seed=16)

In [ ]:
# train_df.coalesce(1).write.mode("overwrite").option("header", True).csv("data/df_train.csv")
# test_df.coalesce(1).write.mode("overwrite").option("header", True).csv("data/df_test.csv")

In [ ]:
train_df.write.mode("overwrite").parquet("data/df_train.parquet")
test_df.write.mode("overwrite").parquet("data/df_test.parquet")

In [ ]:
test_df = spark.read.parquet("data/df_test.parquet").toPandas()
test_df.to_parquet("data/df_test.parquet.gzip")